In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import train_test_split

from nltk.tokenize import sent_tokenize
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# import warnings filter
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import import_ipynb
import data
import re, csv

targetSentiment = ["positive", "negative","neutral"]

array = data.dataCleaning();
tweetsData = array[0]
sentiment = array[1]


# data.createBalancedFile() #balanced unclean file
print('done')

In [20]:
# Creating DATAFRAME

balancedUncleanfile = 'BalancedUncleanFile.csv'
# balancedUncleanfile = 'Tweets.csv'
df = pd.read_csv(balancedUncleanfile)

# df['punctuation'] = df['text'].apply(countPunctuation)
# anger or frustation is generally expressed using upper keywords
df['upper'] = df['text'].apply(lambda i: len([i for i in i.split() if i.isupper()]))
# df['SentimentScore_Vader'] = df['text'].apply(sentimentScore_Vader)
# df['hashtag'] = df['text'].apply(hashtag)
df.to_csv(balancedUncleanfile)
df
print('done')

print(df.loc[1:10,['text','punctuation','upper', 'hashtag', 'SentimentScore_Vader' ]])
df.columns

In [147]:
# Splitting DataSet for Training and Testing
X = df.loc[:,['text', 'upper']]
# X = df.loc[:,['punctuation', 'upper','SentimentScore_Vader', 'tweetVector']]
Y = df.loc[:,['airline_sentiment']]
X_train, X_test, Y_train, Y_test = train_test_split(X,Y , test_size =0.3)
print('Splitting Data complete')
print(X_train.values)

print('done')

print('Y_train:', Y_train.shape)
print('X_test:',X_test.shape)

Splitting Data complete
[["@SouthwestAir pleaseeee resume direct flights from FLL to PHL. I don't want to fly with anyone else 😩😫😢"
  3]
 ["@VirginAmerica I have an unused ticket but moved to a new city where you don't fly. How can I fly with you before it expires? #travelhelp"
  2]
 ['@SouthwestAir prove it' 0]
 ...
 ['@SouthwestAir do you ever reinstate Cancelled Flighted flights?' 0]
 ['@AmericanAir My ex-boyfriend picks up my calls more than you do! #Cancelled Flightedflight #nocustomerservice #americanairlines #FAIL'
  1]
 ['@USAirways an award travel ticket but cannot since it is under a different last name. I simply need to change it'
  1]]
done
Y_train: (4962, 1)
X_test: (2127, 2)


done


In [ ]:
# print(vectorizer.fit_transform(df['text']).toarray().shape)
# df['tweetVector'] = list(vectorizer.fit_transform(df['text']).toarray())
# print('done')
# df.head()

In [175]:
# X_TRAIN DATA
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    analyzer='word', 
    sublinear_tf=True,
    strip_accents='unicode',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 1),
    max_features=10000)

features = vectorizer.fit_transform(X_train.loc[:,'text'])
print(vectorizer)
feature_names = vectorizer.get_feature_names()
print(feature_names)

# #Splitting features into columns
# features_df = pd.DataFrame(features.todense(), columns = feature_names)
# features_df.head()

# # adding UPPER to vector
# features_df['upper'] = X_train['upper']
# features_df.head()

# # removing NaN from upper column
# features_df = features_df.fillna(0)
# features_df.head()

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents='unicode', sublinear_tf=True,
        token_pattern='\\w{1,}', tokenizer=None, use_idf=True,
        vocabulary=None)
['0', '00', '000', '000114', '000lbs', '0011', '00am', '00p', '0167560070877', '01ldxn3qqq', '04sdytt7zd', '05', '0510', '0530', '05am', '05pm', '07p', '08', '0hxlnvzknp', '0jjt4x3yxg', '0jutcdrljl', '0liwecasoe', '0pdntgbxc6', '0xjared', '1', '10', '100', '1000', '1007', '1008', '101', '1016', '1024', '1025', '1027', '103', '1038', '1050', '1051', '1058', '1065', '1071', '1079871763', '1081', '108639', '1098', '1099', '10hrs', '10m', '10min', '10p', '10pm', '10th', '10tmthvfdc', '10voucherwhatajoke', '10x9x17', '11', '110', '1120am', '1136', '1137', '

In [171]:
# X_TEST DATA
features_test = vectorizer.transform(X_test.loc[:,'text'])
features_test_names =  vectorizer.get_feature_names()

#Splitting features into columns
features_test_df = pd.DataFrame(features_test.todense(), columns =features_test_names)
features_test_df.head()

# adding UPPER to vector
features_test_df['upper'] = X_test['upper']
features_test_df.head()

# removing NaN from upper column
features_test_df = features_test_df.fillna(0)
features_test_df.head()

,0,00,000,000114,000lbs,0011,00am,00p,0167560070877,01ldxn3qqq,...,zofwldqxbe,zone,zpz78poeon,zqutus7epw,zrh_airport,zsuztnaijq,ztrdwv0n4l,zurich,ʖ,upper
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
clf = RandomForestClassifier(random_state=0)
clf.fit(features_df.iloc[:, 1000:], Y_train.values.ravel())
predicted = clf.predict(features_test_df.iloc[:, 1000:])
accuracy = accuracy_score(Y_test, predicted)
print(accuracy)

In [174]:
classifiers(features_df, Y_train.values.ravel(), features_test_df,Y_test)

KeyboardInterrupt: 

In [84]:
import pickle

pickle.dump(vectorizer, open("tfidf.pickle", "wb"))
pickle.dump(features, open("train_comment_features.pickle", "wb"))

In [ ]:
#All Classifiers
def classifiers(X_train, Y_train, X_test,Y_test):

    from sklearn.metrics import accuracy_score, log_loss
    from sklearn.svm import SVC
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.linear_model import LogisticRegression
    import pandas as pd
    import seaborn as sns
    import matplotlib.pyplot as plt
    # import warnings filter
    from warnings import simplefilter
    simplefilter(action='ignore', category=FutureWarning)
    
    accList =[]
    clfNameList = []
    

    classifiers = [
        MultinomialNB(), doesnt work for negative values
        SVC(kernel="rbf", C=0.025, probability=True),
        DecisionTreeClassifier(),
        RandomForestClassifier(),
        LogisticRegression(random_state=0)
    ]

    evaluationArr =["Classifier", "Accuracy"]
    log = pd.DataFrame(columns=evaluationArr)

    for clf in classifiers:
        clf.fit(X_train, Y_train)
        clfName = clf.__class__.__name__
        print(clfName)
        clfNameList.append(clfName)

        predicted = clf.predict(X_test)
        accuracy = accuracy_score(Y_test, predicted)
        print("Accuracy: {:.4%}".format(accuracy))
        accList.append(accuracy)
        print(metrics.classification_report(Y_test, predicted, targetSentiment))
        print("="*30)

    #     train_predictions = clf.predict_proba(X_test)[:,3]
    #     ll = log_loss(Y_test, train_predictions)
    #     print("Log Loss: {}".format(ll))

        modelResult = pd.DataFrame([[clfName, accuracy*100]], columns=evaluationArr)
        log = log.append(modelResult)
        
    

    sns.set_color_codes("muted")
    sns.barplot(x='Accuracy', y='Classifier', data=log, color="b")

    plt.xlabel('Accuracy %')
    plt.title('Classifier Accuracy')
    plt.show()

In [ ]:
classifiers(X_train, Y_train.values.ravel(), X_test,Y_test)

In [32]:
from nltk.tokenize import sent_tokenize
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

text="""Hello Mr. Smith, how are you doing today? The weather is great, and city is awesome.
The sky is pinkish-blue. You shouldn't eat cardboard"""

sentenceArr =sent_tokenize(text)
print(sentence)


analyser = SentimentIntensityAnalyzer()


def sentimentScores(sentenceArr):
    score = analyser.polarity_scores(sentence)
    print(score)

# sentimentScores(sentence[3])
for sentence in sentenceArr:
    sentimentScores(sentence)
    


['Hello Mr. Smith, how are you doing today?', 'The weather is great, and city is awesome.', 'The sky is pinkish-blue.', "You shouldn't eat cardboard"]
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.423, 'pos': 0.577, 'compound': 0.8481}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


In [66]:
def sentimentScore_Vader(text):
    from nltk.tokenize import sent_tokenize
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
    analyser = SentimentIntensityAnalyzer()
    score = analyser.polarity_scores(text)
    return score['compound']

def countPunctuation(text):
    punctuations = "!"
    arr = re.split('(\W)',  text)
    count =0 
    for token in arr:
        if token and token in punctuations:
            count = count+1

    return count

#------HASHTAG------
hastagList = []
hashTagDic = {}

def hashtag(text):
    hashArr = re.findall("#\w+",text)
    for i in hashArr:
        i = i[1:]
        if i not in hastagList:
            hastagList.append(i)
    
    for i in hashArr:
        i = i[1:]
        if i in hashTagDic:
            count = hashTagDic[i]
            hashTagDic[i] = count+1
        else:
            hashTagDic[i] =  1
    
   
    return hashArr


with open('HashtagDictionary.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in hashTagDic.items():
       writer.writerow([key, value])
print('done')
 